In [1]:
import os
from groq import Groq
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

from langchain.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

from llama_index.core.llms import ChatMessage, MessageRole

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnableSequence

from langchain.chains import LLMChain, SequentialChain

In [4]:
load_dotenv(dotenv_path="config.env")
api_key = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = api_key
llm_model = "llama3-70b-8192"

In [5]:
llm = ChatGroq(
    model=llm_model,
    temperature=1,
    max_retries=2,
)
output_dict_list = []

In [111]:
system_prompt ="""You are required to generate domain names based on a given topic. 
    For every request, you will be provided with a topic, and your task 
    is to generate domain names that include the word 'DOMAIN' before 
    each suggested name. Below are examples of how the input request 
    and your response should look like:
    
    Example:
    
    Request: Please help me generate 3 domain names about cooking.
    Response:
    DOMAIN: TastyHub.com
    DOMAIN: RecipeRush.net
    DOMAIN: TheCookingCoach.io"""

user_prompt = "Please help me generate {number} domains related to {topic} with tld {tld}"
prompt_template = ChatPromptTemplate([
    ("system", system_prompt),
    ("user", user_prompt)
])
num = 20
tld = ".com"
topic = "phone"
customer_messages = prompt_template.format_messages(
                    number=20,
                    tld=tld,
                    topic=topic)
print(customer_messages)

[SystemMessage(content="You are required to generate domain names based on a given topic. \n    For every request, you will be provided with a topic, and your task \n    is to generate domain names that include the word 'DOMAIN' before \n    each suggested name. Below are examples of how the input request \n    and your response should look like:\n    \n    Example:\n    \n    Request: Please help me generate 3 domain names about cooking.\n    Response:\n    DOMAIN: TastyHub.com\n    DOMAIN: RecipeRush.net\n    DOMAIN: TheCookingCoach.io", additional_kwargs={}, response_metadata={}), HumanMessage(content='Please help me generate 20 domains related to phone with tld .com', additional_kwargs={}, response_metadata={})]


In [112]:
response1 = chat(customer_messages)

In [113]:
print(response1.content)

Here are 20 domain name suggestions related to phones with the TLD .com:

1. DOMAIN: PhoneFiesta.com
2. DOMAIN: MobileMasters.com
3. DOMAIN: CellSavvy.com
4. DOMAIN: PhoneProPlus.com
5. DOMAIN: SmartPhoneHub.com
6. DOMAIN: RingMyBell.com
7. DOMAIN: PhoneBuzzing.com
8. DOMAIN: MobileMoments.com
9. DOMAIN: PhoneFusion.com
10. DOMAIN: CellPhoneSavvy.com
11. DOMAIN: DialDirectly.com
12. DOMAIN: PhonePitStop.com
13. DOMAIN: MobileMatey.com
14. DOMAIN: PhoneFanatic.com
15. DOMAIN: CellCentral.com
16. DOMAIN: PhonePhoria.com
17. DOMAIN: Mobile Marvels.com
18. DOMAIN: PhoneGenie.com
19. DOMAIN: CellConnection.com
20. DOMAIN: PhonePresto.com

Let me know if you need more!


In [114]:
domain_schema = ResponseSchema(
    name="domain",
    description="""\
    Was the domain name of the text \
    For example, if the text contains The domains are "hello.com and byebye.net"  \
    the output should be a list: ["hello.com", "byebye.net"].
    """
)
response_schemas = [domain_schema,]

In [115]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [116]:
format_instructions = output_parser.get_format_instructions()

In [117]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"domain": string  //     Was the domain name of the text     For example, if the text contains The domains are "hello.com and byebye.net"      the output should be a list: ["hello.com", "byebye.net"].
    
}
```


In [129]:
review_template1 = """\
For the given text, extract the following information:

- **domain**: Identify all domain names mentioned in the text and return them as a list. 

**Text To Analyze:** 
{text}
**End Of Text**
{format_instructions}

Please output the result strictly in valid JSON format, without any additional text, commentary, or explanation.

"""

review_prompt = ChatPromptTemplate.from_template(template=review_template1)

messages = review_prompt.format_messages(text=response1.content, 
                                format_instructions=format_instructions)

In [130]:
print(messages[0].content)

For the given text, extract the following information:

- **domain**: Identify all domain names mentioned in the text and return them as a list. 

**Text To Analyze:** 
Here are 20 domain name suggestions related to phones with the TLD .com:

1. DOMAIN: PhoneFiesta.com
2. DOMAIN: MobileMasters.com
3. DOMAIN: CellSavvy.com
4. DOMAIN: PhoneProPlus.com
5. DOMAIN: SmartPhoneHub.com
6. DOMAIN: RingMyBell.com
7. DOMAIN: PhoneBuzzing.com
8. DOMAIN: MobileMoments.com
9. DOMAIN: PhoneFusion.com
10. DOMAIN: CellPhoneSavvy.com
11. DOMAIN: DialDirectly.com
12. DOMAIN: PhonePitStop.com
13. DOMAIN: MobileMatey.com
14. DOMAIN: PhoneFanatic.com
15. DOMAIN: CellCentral.com
16. DOMAIN: PhonePhoria.com
17. DOMAIN: Mobile Marvels.com
18. DOMAIN: PhoneGenie.com
19. DOMAIN: CellConnection.com
20. DOMAIN: PhonePresto.com

Let me know if you need more!
**End Of Text**
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```jso

In [131]:
response = chat(messages)

In [132]:
print(response.content)

```json
{
	"domain": ["PhoneFiesta.com", "MobileMasters.com", "CellSavvy.com", "PhoneProPlus.com", "SmartPhoneHub.com", "RingMyBell.com", "PhoneBuzzing.com", "MobileMoments.com", "PhoneFusion.com", "CellPhoneSavvy.com", "DialDirectly.com", "PhonePitStop.com", "MobileMatey.com", "PhoneFanatic.com", "CellCentral.com", "PhonePhoria.com", "Mobile Marvels.com", "PhoneGenie.com", "CellConnection.com", "PhonePresto.com"]
}
```


In [133]:
output_dict = output_parser.parse(response.content)

In [134]:
output_dict

{'domain': ['PhoneFiesta.com',
  'MobileMasters.com',
  'CellSavvy.com',
  'PhoneProPlus.com',
  'SmartPhoneHub.com',
  'RingMyBell.com',
  'PhoneBuzzing.com',
  'MobileMoments.com',
  'PhoneFusion.com',
  'CellPhoneSavvy.com',
  'DialDirectly.com',
  'PhonePitStop.com',
  'MobileMatey.com',
  'PhoneFanatic.com',
  'CellCentral.com',
  'PhonePhoria.com',
  'Mobile Marvels.com',
  'PhoneGenie.com',
  'CellConnection.com',
  'PhonePresto.com']}

In [137]:
output_dict['input'] = customer_messages[1].content

In [138]:
print(output_dict)

{'domain': ['PhoneFiesta.com', 'MobileMasters.com', 'CellSavvy.com', 'PhoneProPlus.com', 'SmartPhoneHub.com', 'RingMyBell.com', 'PhoneBuzzing.com', 'MobileMoments.com', 'PhoneFusion.com', 'CellPhoneSavvy.com', 'DialDirectly.com', 'PhonePitStop.com', 'MobileMatey.com', 'PhoneFanatic.com', 'CellCentral.com', 'PhonePhoria.com', 'Mobile Marvels.com', 'PhoneGenie.com', 'CellConnection.com', 'PhonePresto.com'], 'input': 'Please help me generate 20 domains related to phone with tld .com'}


In [140]:
output_dict_list.append(output_dict)

In [141]:
output_dict_list[0]

{'domain': ['PhoneFiesta.com',
  'MobileMasters.com',
  'CellSavvy.com',
  'PhoneProPlus.com',
  'SmartPhoneHub.com',
  'RingMyBell.com',
  'PhoneBuzzing.com',
  'MobileMoments.com',
  'PhoneFusion.com',
  'CellPhoneSavvy.com',
  'DialDirectly.com',
  'PhonePitStop.com',
  'MobileMatey.com',
  'PhoneFanatic.com',
  'CellCentral.com',
  'PhonePhoria.com',
  'Mobile Marvels.com',
  'PhoneGenie.com',
  'CellConnection.com',
  'PhonePresto.com'],
 'input': 'Please help me generate 20 domains related to phone with tld .com'}

In [8]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
output_parser = StrOutputParser()
# chain 1: input= Review and output= English_Review
English_review = first_prompt | llm | output_parser


In [9]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

summary = second_prompt | llm | output_parser

In [11]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
language = third_prompt | llm | output_parser

In [13]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
followup_message = fourth_prompt | llm | output_parser

In [25]:
pipeline = RunnableSequence([
    {"english_review": English_review},
    {"summary": summary},
    {"language": language},
    {"followup_message": followup_message},
])

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>